In [1]:
from nile.api.v1.clusters import Hahn as HahnYql

In [2]:
from nile.api.v1.clusters.yt import Hahn as HahnYt

In [3]:
import qb2.api.v1.filters as sf
from nile.api.v1 import extractors as ne

In [4]:
from tqdm.notebook import tqdm

In [5]:
cluster = HahnYt(pool='alpaca')

In [6]:
date = '2020-03-24'

In [7]:
[
    x for x in 
    tqdm(cluster.read('//home/market/production/replenishment/order_planning/latest/intermediate/suppliers'))
    if x.rs_id=='003222'
]

[Record(rs_id='003222', supplier_id=561382, supplier_type=1)]

In [8]:
pilot_suppliers = {481645}

In [9]:
from qb2.api.v1 import extractors as se
from qb2.api.v1 import filters as sf

In [10]:
job = cluster.job().env(parallel_operations_limit=10)
tmp = (
    job.table('//home/market/production/replenishment/order_planning/{}/intermediate/assortment'.format(date))
    .qb2(
        log='generic-yson-log',
        fields=[
            se.log_field('supplier_id').hide(),
            se.log_field('msku'),
        ],
        filters=[
            sf.one_of('supplier_id', pilot_suppliers)
        ],
        intensity='cpu'
    )
    .put('//tmp/astepanel-sample-assortment1', merge_strategy='never')
)

job.run()



VBox()

In [11]:
pilot_mskus=set([
    x.msku for x in 
    tqdm(cluster.read('//tmp/astepanel-sample-assortment1'))
    ]
)

In [12]:
len(pilot_mskus)

405

In [13]:
len(pilot_suppliers)

1

In [14]:
job = cluster.job().env(parallel_operations_limit=10)

In [15]:
orders = (
    job.table('//home/market/production/replenishment/order_planning/{}/intermediate/orders_alpaca'.format(date))
    .filter(sf.one_of('msku', pilot_mskus))
    .put('//home/market/development/replenishment/denisk/test1/orders_alpaca', merge_strategy = 'never')
)

In [16]:
prices = (
    job.table('//home/market/production/replenishment/order_planning/{}/intermediate/prices_alpaca'.format(date))
    .filter(sf.one_of('msku', pilot_mskus))
    .put('//home/market/development/replenishment/denisk/test1/prices_alpaca', merge_strategy = 'never')
)

In [17]:
transits = (
    job.table('//home/market/production/replenishment/order_planning/{}/outputs/transits'.format(date))
    .filter(sf.one_of('msku', pilot_mskus))
    .put('//home/market/development/replenishment/denisk/test1/transits', merge_strategy = 'never')
)

In [18]:
stock = (
    job.table('//home/market/production/replenishment/order_planning/{}/intermediate/stock_alpaca'.format(date))
    .filter(sf.one_of('msku', pilot_mskus))
    .put('//home/market/development/replenishment/denisk/test1/stock_alpaca', merge_strategy = 'never')
)

In [19]:
forecast  =(
    job.table('//home/market/production/replenishment/order_planning/{}/intermediate/forecast_alpaca'.format(date))
    .filter(sf.one_of('msku', pilot_mskus))
    .put('//home/market/development/replenishment/denisk/test1/forecast_alpaca', merge_strategy = 'never')
)

In [20]:
ss = (
    job.table('//home/market/production/replenishment/order_planning/{}/intermediate/ss'.format(date))
    .filter(sf.one_of('msku', pilot_mskus))
    .put('//home/market/development/replenishment/denisk/test1/ss', merge_strategy = 'never')
)

In [21]:
delivery_options= (
    job.table('//home/market/production/replenishment/order_planning/{}/intermediate/delivery_options'.format(date))
    .filter(sf.one_of('supplier_id', pilot_suppliers))
    .put('//home/market/development/replenishment/denisk/test1/delivery_options', merge_strategy = 'never')
)

In [22]:
msku_delivery_options = (
    job.table('//home/market/production/replenishment/order_planning/{}/intermediate/msku_delivery_options'.format(date))
    .filter(sf.one_of('msku', pilot_mskus))
    .put('//home/market/development/replenishment/denisk/test1/msku_delivery_options', merge_strategy = 'never')
)

In [23]:
assortment = (
    job.table('//home/market/production/replenishment/order_planning/{}/intermediate/assortment'.format(date))
    .filter(sf.one_of('msku', pilot_mskus))
    .put('//home/market/development/replenishment/denisk/test1/assortment', merge_strategy = 'never')
)

In [24]:
warehouses = (
    job.table('//home/market/production/replenishment/order_planning/latest/inputs/warehouses_raw')
    .put('//home/market/development/replenishment/denisk/test1/warehouses', merge_strategy = 'never')
)

In [25]:
suppliers = (
    job.table('//home/market/production/replenishment/order_planning/latest/intermediate/suppliers')
    .project(
        ne.all(),
        date_abs_errs=ne.const({"0": 1.0}),
        qty_rel_errs=ne.const({"0": 1.0})
    )
    .put('//home/market/development/replenishment/denisk/test1/suppliers', merge_strategy = 'never')
)

In [26]:
from nile.api.v1 import aggregators as na

In [27]:
from nile.api.v1 import extractors as ne

In [28]:
# Закладка для SN по генерации семла весогабаритов
# dimensions = (
#     job.table('//home/market/production/mstat/analyst/regular/cubes_vertica/dim_aproximate_weight_and_dimensions')
#     .project('msku','volume')
# #     .filter(sf.one_of('msku', pilot_mskus))
#     .groupby('msku')
#     .aggregate(volume = na.mean('volume'))
#     .put(
#         '//home/market/development/replenishment/astepanel/alpaca_backtest_sample/intermediate/dimensions', 
#         merge_strategy = 'never',
#         schema={
#             'msku':int,
#             'volume':float
#         }
#     )
# )

In [29]:
job.run()

VBox()

In [30]:
!yt read-table --format '<format=pretty>yson' //home/market/development/replenishment/denisk/test1/orders_alpaca > orders_alpaca.yson

In [31]:
!yt read-table --format '<format=pretty>yson' //home/market/development/replenishment/denisk/test1/prices_alpaca > prices_alpaca.yson

2020-03-31 12:54:09,826	INFO	Table chunks are too small; consider running the following command to improve read performance: yt merge --proxy hahn.yt.yandex.net --src //home/market/development/replenishment/denisk/test1/prices_alpaca --dst //home/market/development/replenishment/denisk/test1/prices_alpaca --spec '{combine_chunks=true;}'


In [32]:
!yt read-table --format '<format=pretty>yson' //home/market/development/replenishment/denisk/test1/transits > transits.yson

In [33]:
!yt read-table --format '<format=pretty>yson' //home/market/development/replenishment/denisk/test1/stock_alpaca > stock_alpaca.yson

In [34]:
!yt read-table --format '<format=pretty>yson' //home/market/development/replenishment/denisk/test1/forecast_alpaca > forecast_alpaca.yson

In [35]:
!yt read-table --format '<format=pretty>yson' //home/market/development/replenishment/denisk/test1/ss > ss.yson

In [36]:
!yt read-table --format '<format=pretty>yson' //home/market/development/replenishment/denisk/test1/delivery_options > delivery_options.yson

In [37]:
!yt read-table --format '<format=pretty>yson' //home/market/development/replenishment/denisk/test1/msku_delivery_options > msku_delivery_options.yson

In [38]:
!yt read-table --format '<format=pretty>yson' //home/market/development/replenishment/denisk/test1/assortment > assortment.yson

In [39]:
!yt read-table --format '<format=pretty>yson' //home/market/development/replenishment/denisk/test1/warehouses > warehouses.yson

In [40]:
!yt read-table --format '<format=pretty>yson' //home/market/development/replenishment/denisk/test1/suppliers > suppliers.yson

In [41]:
!head -10 suppliers.yson

{
    "supplier_id" = 460741;
    "rs_id" = #;
    "supplier_type" = 3;
};
{
    "supplier_id" = 462656;
    "rs_id" = #;
    "supplier_type" = 3;
};


In [42]:
!head -10 warehouses.yson

{
    "id" = 4;
    "name" = "\xD0\xA1\xD0\xA6 \xD0\xA1\xD0\x94\xD0\xA2";
    "type_field" = "sorting_center";
};
{
    "id" = 131;
    "name" = "\xD0\x9C\xD0\xB0\xD1\x80\xD1\x88\xD1\x80\xD1\x83\xD1\x82";
    "type_field" = "sorting_center";
};
